In [1]:
import json
from glob import glob
import fnmatch
from multiprocessing import Pool
from pathlib import Path
import os, itertools
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from pickle import dumps, loads, dump, load

from stexls import *

In [2]:
%rm /tmp/linker.bin

rm: cannot remove '/tmp/linker.bin': No such file or directory


In [3]:
root = Path.home()/'MathHub'

In [4]:
try:
    with open('/tmp/linker.bin', 'rb') as fd:
        linker = load(fd)
except Exception as e:
    print(e)
    linker = Linker(root=root, file_pattern='**/*.tex')
    print(linker.update(progress=tqdm, use_multiprocessing=True))
    with open('/tmp/linker.bin', 'w+b') as fd:
        dump(linker, fd)


[Errno 2] No such file or directory: '/tmp/linker.bin'


/home/marian/MathHub/MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
Undefined module: "symtax-base/MODULE" not defined in "/home/marian/MathHub/MiKoMH/GenCS/source/logic/en/syntax-base.tex"
Undefined module: "dl-ontology/MODULE" not defined in "/home/marian/MathHub/MiKoMH/CompLog/source/kr/en/ontology.tex"
Undefined module: "aboxes/MODULE" not defined in "/home/marian/MathHub/MiKoMH/CompLog/source/kr/en/aboxes.tex"
/home/marian/MathHub/MiKoMH/GenCS/source/course/de/CS-sample-performance.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/time.tex"
/home/marian/MathHub/MiKoMH/GenCS/source/course/de/CS-sample-performance.tex:3:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/siunits-prefixed.tex"
/home/marian/MathHub/MiKoMH/CompLog/source/stlc/en/lambda-calc-syntax.tex:6:1: File not indexed:"/home/marian/MathHub/smglom/smglom/source/alpharenaming.tex"
/

/home/marian/MathHub/MiKoMH/talks/source/formalcad/en/flange-ontology.tex:2:1: File not indexed:"/home/marian/MathHub/KwarcMH/FormalCAD/source/flange/cds/ISOmetricthread.tex"
/home/marian/MathHub/MiKoMH/GenCS/source/course/en/CS-sample-performance.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/time.tex"
/home/marian/MathHub/MiKoMH/GenCS/source/course/en/CS-sample-performance.tex:3:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/siunits-prefixed.tex"
Undefined module: "library-exports/MODULE" not defined in "/home/marian/MathHub/MiKoMH/talks/source/oaf/en/library-exports.tex"
Undefined module: "physml-observables/MODULE" not defined in "/home/marian/MathHub/MiKoMH/talks/source/physml/en/nutshell.tex"
Undefined module: "dtl-ago/MODULE" not defined in "/home/marian/MathHub/MiKoMH/AI/source/ml/en/dtl-algo.tex"
Undefined module: "python-nutshell/MODULE" not defined in "/home/marian/MathHub/MiKoMH/GenICT/source/programming/en/genict-py

In [5]:
%ls -ahl /tmp/linker.bin

-rw-rw-r-- 1 marian marian 69M Apr  9 16:17 /tmp/linker.bin


In [6]:
for os in linker.objects.values():
    for o in os:
        for l, es in o.errors.items():
            for e in es:
                print(l.format_link())
                print('\t', e)
                print()
                print('-'*80)
                print()

/home/marian/MathHub/MiKoMH/IWGS/source/webapps/en/forms-recap.tex:19:34
	 Trefi argument count mismatch: Expected 1 vs. actual 2.

--------------------------------------------------------------------------------

/home/marian/MathHub/MiKoMH/IWGS/source/webapps/en/forms-recap.tex:51:6
	 Trefi argument count mismatch: Expected 1 vs. actual 2.

--------------------------------------------------------------------------------

/home/marian/MathHub/MiKoMH/IWGS/source/webapps/en/forms-recap.tex:53:3
	 Trefi argument count mismatch: Expected 1 vs. actual 2.

--------------------------------------------------------------------------------

/home/marian/MathHub/MiKoMH/AI/source/csp/en/mrv.tex:9:58
	 Duplicate symbol definition of mrv.MRV/SYMBOL previously defined at "/home/marian/MathHub/MiKoMH/AI/source/csp/en/mrv.tex:9:11"

--------------------------------------------------------------------------------

/home/marian/MathHub/MiKoMH/GenCS/source/codes/en/unicode-ucs.tex:29:63
	 Duplicate symbo

/home/marian/MathHub/MiKoMH/AI/source/ml/en/MDL.tex:33:7
	 Duplicate symbol definition of MDL.MDL-hypothesis/SYMBOL previously defined at "/home/marian/MathHub/MiKoMH/AI/source/ml/en/MDL.tex:32:11"

--------------------------------------------------------------------------------

/home/marian/MathHub/MiKoMH/talks/source/simsalabim/fragments/twomotivations.tex:15:9
	 Invalid environment of type Defi in file.

--------------------------------------------------------------------------------

/home/marian/MathHub/MiKoMH/TDM/source/digdocs/en/epub.tex:22:73
	 Duplicate symbol definition of epub.IDPF/SYMBOL previously defined at "/home/marian/MathHub/MiKoMH/TDM/source/digdocs/en/epub.tex:22:11"

--------------------------------------------------------------------------------

/home/marian/MathHub/MiKoMH/AI/source/search/en/blackwhitebox.tex:19:7
	 Duplicate symbol definition of blackwhitebox.declarative-description/SYMBOL previously defined at "/home/marian/MathHub/MiKoMH/AI/source/search/en

In [7]:
path = root / Path('MiKoMH/talks/source/sTeX/en/sTeX-approach.tex')

In [8]:
linker.update(tqdm)

{}

In [9]:
linker.view_import_graph(path)

In [10]:
for id, symbols in o.symbol_table.items():
    for s in symbols:
        print(id, s.definition_type, type(s.definition_type))

In [11]:
for o in linker.get_relevant_objects(path, 1, 40):
    print(o.__dict__)
    print(o.format())

{'root': PosixPath('/home/marian/MathHub'), 'files': {PosixPath('/home/marian/MathHub/smglom/sets/source/transitive-closure.tex'), PosixPath('/home/marian/MathHub/MiKoMH/GenICT/source/python/en/variables.tex'), PosixPath('/home/marian/MathHub/smglom/computing/source/storage-medium.tex'), PosixPath('/home/marian/MathHub/MiKoMH/GenICT/source/python/en/datatypes.tex'), PosixPath('/home/marian/MathHub/smglom/sets/source/fundefeq.tex'), PosixPath('/home/marian/MathHub/smglom/computing/source/code.tex'), PosixPath('/home/marian/MathHub/smglom/sets/source/symmetric.tex'), PosixPath('/home/marian/MathHub/MiKoMH/GenCS/source/codes/en/character-codes.tex'), PosixPath('/home/marian/MathHub/smglom/sets/source/emptyset.tex'), PosixPath('/home/marian/MathHub/MiKoMH/GenICT/source/python/en/nutshell.tex'), PosixPath('/home/marian/MathHub/smglom/sets/source/multirel.tex'), PosixPath('/home/marian/MathHub/MiKoMH/TDM/source/doccomp/en/sed.tex'), PosixPath('/home/marian/MathHub/smglom/sets/source/cartesia

In [12]:
o.find_mhmodule_module(
    current_file=path,
    context='',
    mhrepo='MiKoMH/TDM',
    path='LaTeX/en/tex-packages')

[tex-packages/MODULE,
 AMSTeX/SYMBOL,
 bibTeX/SYMBOL,
 TIKZ/SYMBOL,
 LaTeXformat/SYMBOL,
 pdflatexprogram/SYMBOL]